다음 데이터 세트를 이용하여 다이아몬드 가격예측(회귀) 

url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv" diamonds = pd.read_csv(url) 

1. ML (RF, DT, LR) 수행

2. Dense layer만 이용 FNCC 구현

3. 순환 데이터 변환후 CNN 구현



각 단계별로 수행완료후

검사받은 이후에 다음 단계 진행

In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

url='https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
df = pd.read_csv(url)
df

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc
...,...,...,...,...,...,...,...
1722,low,low,5more,more,med,med,good
1723,low,low,5more,more,med,high,vgood
1724,low,low,5more,more,big,low,unacc
1725,low,low,5more,more,big,med,good


In [3]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['vhigh'] = label_encoder.fit_transform(df['vhigh'])
label_encoder = LabelEncoder()
df['vhigh.1'] = label_encoder.fit_transform(df['vhigh.1'])
label_encoder = LabelEncoder()
df['2'] = label_encoder.fit_transform(df['2'])
label_encoder = LabelEncoder()
df['2.1'] = label_encoder.fit_transform(df['2.1'])
label_encoder = LabelEncoder()
df['small'] = label_encoder.fit_transform(df['small'])
label_encoder = LabelEncoder()
df['low'] = label_encoder.fit_transform(df['low'])
label_encoder = LabelEncoder()
df['unacc'] = label_encoder.fit_transform(df['unacc'])

In [5]:
X=df.drop('unacc',axis=1)
X.head()

,vhigh,vhigh.1,2,2.1,small,low
0,3,3,0,0,2,2
1,3,3,0,0,2,0
2,3,3,0,0,1,1
3,3,3,0,0,1,2
4,3,3,0,0,1,0


In [6]:
y=df['unacc']
y.value_counts()

unacc
2    1209
0     384
1      69
3      65
Name: count, dtype: int64

In [7]:
y.head()

0    2
1    2
2    2
3    2
4    2
Name: unacc, dtype: int64

In [8]:
Y = pd.get_dummies(y).values

In [9]:
X = X.values

In [11]:
X

array([[3, 3, 0, 0, 2, 2],
       [3, 3, 0, 0, 2, 0],
       [3, 3, 0, 0, 1, 1],
       ...,
       [1, 1, 3, 2, 0, 1],
       [1, 1, 3, 2, 0, 2],
       [1, 1, 3, 2, 0, 0]])

In [12]:
Y

array([[False, False,  True, False],
       [False, False,  True, False],
       [False, False,  True, False],
       ...,
       [False, False,  True, False],
       [False,  True, False, False],
       [False, False, False,  True]])

In [18]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.25,random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1295, 6) (432, 6) (1295, 4) (432, 4)


In [27]:
def split_sequences(sequences,n_features, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [30]:
# 입력 데이터 형식 확인
print(f"Original X_train shape: {X_train.shape}")
print(f"Original X_test shape: {X_test.shape}")

# 입력 데이터 형식 변경
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(f"Reshaped X_train shape: {X_train.shape}")
print(f"Reshaped X_test shape: {X_test.shape}")

Original X_train shape: (1295, 6)
Original X_test shape: (432, 6)
Reshaped X_train shape: (1295, 1, 6)
Reshaped X_test shape: (432, 1, 6)


In [31]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adam

from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Input
from keras.layers import Dense, Flatten, Dropout

In [32]:
import tensorflow as tf

n_features = 6
n_timesteps = 1

input_layer = Input(shape=(n_timesteps,n_features))

conv1 = Conv1D(filters=64, kernel_size=1, activation='relu')(input_layer) 
flat = Flatten()(conv1)
dens = Dense(64, activation = "relu")(flat)
outputs = Dense(4, activation = "sigmoid")(dens)

model = keras.Model(input_layer, outputs)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1, 6)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 1, 64)          │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,868 (19.02 KB)

 Trainable params: 4,868 (19.02 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
history=model.fit(x=X_train, y=y_train, epochs=30, batch_size=32,validation_data= (X_test,y_test))
y_pred = model.predict(X_test)

y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

Epoch 1/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5530 - loss: 0.5623 - val_accuracy: 0.6898 - val_loss: 0.3789
Epoch 2/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step - accuracy: 0.7107 - loss: 0.3503 - val_accuracy: 0.6898 - val_loss: 0.3425
Epoch 3/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - accuracy: 0.7176 - loss: 0.3209 - val_accuracy: 0.6898 - val_loss: 0.3197
Epoch 4/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - accuracy: 0.7259 - loss: 0.3006 - val_accuracy: 0.6991 - val_loss: 0.3069
Epoch 5/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step - accuracy: 0.7156 - loss: 0.2866 - val_accuracy: 0.7014 - val_loss: 0.2967
Epoch 6/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - accuracy: 0.7372 - loss: 0.2743 - val_accuracy: 0.7037 - val_loss: 0.2867
Epoch 7/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - accuracy: 0.7261 - loss: 0.2670 - val_accuracy: 0.7245 - val_loss: 0.2751
Epoch 8/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step - accuracy: 0.7489 - loss: 0.2514 - val_accuracy: 0.72